In [582]:
import pandas as pd
import re
from sqlalchemy import create_engine

# Extracting Data

## Individual cyber attacks

In [583]:
# Read the CSV file
df_individuals = pd.read_csv('../Original Datasets/individuals_cyber_attacks_europe.csv')
df_ind = df_individuals.copy()
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [584]:
# Database connection details
db_user = 'root'  # Replace with your MySQL username
db_password = 'root'  # Replace with your MySQL password
db_host = 'localhost'  # Or your database server hostname
db_name = 'staging_cyber_attacks'  # Replace with your database name
table_name = 'cyber_attacks_individuals'  # Replace with your desired table name

# Create the database engine
engine = create_engine(f'mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}')

# Write the DataFrame to the MySQL table
df_individuals.to_sql(table_name, con=engine, if_exists='replace', index=False)

print(f"Data imported successfully to table '{table_name}' in database '{db_name}'")

Data imported successfully to table 'cyber_attacks_individuals' in database 'staging_cyber_attacks'


## Companies cyber attacks

In [585]:
# Read the CSV file
df_companies = pd.read_csv('../Original Datasets/companies_cyber_attacks_europe.csv')
df_comp = df_companies.copy()
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [586]:
# Database connection details
db_user = 'root'  # Replace with your MySQL username
db_password = 'root'  # Replace with your MySQL password
db_host = 'localhost'  # Or your database server hostname
db_name = 'staging_cyber_attacks'  # Replace with your database name
table_name = 'cyber_attacks_companies'  # Replace with your desired table name

# Create the database engine
engine = create_engine(f'mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}')

# Write the DataFrame to the MySQL table
df_companies.to_sql(table_name, con=engine, if_exists='replace', index=False)

print(f"Data imported successfully to table '{table_name}' in database '{db_name}'")

Data imported successfully to table 'cyber_attacks_companies' in database 'staging_cyber_attacks'


## Active Users

In [587]:
# Read the CSV file
df_users = pd.read_csv('../Original Datasets/active_online_users_europe_2018_2023.csv')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [588]:
# Database connection details
db_user = 'root'  # Replace with your MySQL username
db_password = 'root'  # Replace with your MySQL password
db_host = 'localhost'  # Or your database server hostname
db_name = 'staging_cyber_attacks'  # Replace with your database name
table_name = 'active_online_users'  # Replace with your desired table name

# Create the database engine
engine = create_engine(f'mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}')

# Write the DataFrame to the MySQL table
df_users.to_sql(table_name, con=engine, if_exists='replace', index=False)

print(f"Data imported successfully to table '{table_name}' in database '{db_name}'")

Data imported successfully to table 'active_online_users' in database 'staging_cyber_attacks'


# Transforming Data

## Cyber Attacks on Individuals

In [589]:
# Drop rows where all elements are missing
df_ind.dropna(how='all', inplace=True)

In [590]:
df_ind['Timestamp'] = pd.to_datetime(df_ind['Timestamp'], format='%m/%d/%Y %H:%M')

In [591]:
# Fill nulls with 'Not Available'
columns_to_fill = ['Malware Indicators', 'Alerts/Warnings', 'Proxy Information', 'Firewall Logs', 'IDS/IPS Alerts']
df_ind[columns_to_fill] = df_ind[columns_to_fill].fillna('Not Available')

In [592]:
# Drop duplicates
df_ind.drop_duplicates(inplace=True)
df_ind.reset_index(drop=True, inplace=True)

In [593]:
# Validate IP addresses
def is_valid_ip(ip):
    pattern = r"^((25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.){3}(25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)$"
    return bool(re.match(pattern, ip))

df_ind['Valid Source IP'] = df_ind['Source IP Address'].apply(is_valid_ip)
df_ind['Valid Destination IP'] = df_ind['Destination IP Address'].apply(is_valid_ip)

In [594]:
# Check for invalid port numbers
invalid_source_ports = df_ind[(df_ind['Source Port'] < 0) | (df_ind['Source Port'] > 65535)]
invalid_destination_ports = df_ind[(df_ind['Destination Port'] < 0) | (df_ind['Destination Port'] > 65535)]

if invalid_source_ports.shape[0] > 0 or invalid_destination_ports.shape[0] > 0:
    print('Invalid port numbers found')
else:
    print('All port numbers are valid')

All port numbers are valid


In [595]:
# Count invalid IPs
invalid_source_ip_count = df_ind[df_ind['Valid Source IP'] == False].shape[0]
invalid_destination_ip_count = df_ind[df_ind['Valid Destination IP'] == False].shape[0]

print(f"\nNumber of rows with invalid Source IP Address: {invalid_source_ip_count}")
print(f"Number of rows with invalid Destination IP Address: {invalid_destination_ip_count}")


Number of rows with invalid Source IP Address: 0
Number of rows with invalid Destination IP Address: 0


In [596]:
# Extract features from 'Timestamp'
df_ind['Day'] = df_ind['Timestamp'].dt.day
df_ind['Hour of Day'] = df_ind['Timestamp'].dt.hour
df_ind['Month'] = df_ind['Timestamp'].dt.month
df_ind['Year'] = df_ind['Timestamp'].dt.year

In [597]:
def extract_browser(text):
    # Define patterns for common browsers
    patterns = {
        'Chrome': r'Chrome\/[\d.]+',
        'Firefox': r'Firefox\/[\d.]+',
        'Safari': r'Safari\/[\d.]+',
        'Edge': r'Edge\/[\d.]+',
        'MSIE': r'MSIE [\d.]+',
        'Trident': r'Trident\/[\d.]+'  # For older IE versions
    }

    for browser, pattern in patterns.items():
        match = re.search(pattern, text)
        if match:
            return browser
    return 'Unknown'

In [598]:
def extract_os(text):
    # Define patterns for common operating systems
    patterns = {
        'Windows': r'Windows NT [\d.]+',
        'Mac OS X': r'Mac OS X [\d._]+',
        'Linux': r'Linux',
        'Android': r'Android [\d.]+',
        'iOS': r'iPhone|iPad|iPod'
    }

    for os, pattern in patterns.items():
        match = re.search(pattern, text)
        if match:
            return os
    return 'Unknown'

In [599]:
def extract_device_type(text):
    # Define patterns for common device types
    if 'Mobile' in text or 'Android' in text or 'iPhone' in text or 'iPad' in text or 'iPod' in text:
        return 'Mobile'
    elif 'Tablet' in text:
        return 'Tablet'
    else:
        return 'Desktop'

In [600]:
# Apply the functions to the 'Device Information' column
df_ind['Browser'] = df_ind['Device Information'].astype(str).apply(extract_browser)
df_ind['Operating System'] = df_ind['Device Information'].astype(str).apply(extract_os)
df_ind['Device Type'] = df_ind['Device Information'].astype(str).apply(extract_device_type)

In [601]:
df_ind = df_ind.drop('Device Information', axis=1)

In [602]:
df_ind.head()

,Timestamp,Source IP Address,Destination IP Address,Source Port,Destination Port,Protocol,Packet Length,Packet Type,Traffic Type,Payload Data,Malware Indicators,Anomaly Scores,Alerts/Warnings,Attack Type,Attack Signature,Action Taken,Severity Level,User Information,Network Segment,Proxy Information,Firewall Logs,IDS/IPS Alerts,Log Source,Country,Valid Source IP,Valid Destination IP,Day,Hour of Day,Month,Year,Browser,Operating System,Device Type
0,2023-05-30 06:33:00,103.216.15.12,84.9.164.252,31225,17616,ICMP,503,Data,HTTP,Qui natus odio asperiores nam. Optio nobis ius...,IoC Detected,28.67,Not Available,Malware,Known Pattern B,Logged,Low,Reyansh Dugal,Segment A,150.9.97.135,Log Data,Not Available,Server,Germany,True,True,30,6,5,2023,MSIE,Windows,Desktop
1,2020-08-26 07:08:00,78.199.217.198,66.191.137.154,17245,48166,ICMP,1174,Data,HTTP,Aperiam quos modi officiis veritatis rem. Omni...,IoC Detected,51.50,Not Available,Malware,Known Pattern A,Blocked,Low,Sumer Rana,Segment B,Not Available,Log Data,Not Available,Firewall,Germany,True,True,26,7,8,2020,MSIE,Windows,Desktop
2,2022-11-13 08:23:00,63.79.210.48,198.219.82.17,16811,53600,UDP,306,Control,HTTP,Perferendis sapiente vitae soluta. Hic delectu...,IoC Detected,87.42,Alert Triggered,DDoS,Known Pattern B,Ignored,Low,Himmat Karpe,Segment C,114.133.48.179,Log Data,Alert Data,Firewall,Spain,True,True,13,8,11,2022,MSIE,Windows,Desktop
3,2023-07-02 10:38:00,163.42.196.10,101.228.192.255,20018,32534,UDP,385,Data,HTTP,Totam maxime beatae expedita explicabo porro l...,Not Available,15.79,Alert Triggered,Malware,Known Pattern B,Blocked,Medium,Fateh Kibe,Segment B,Not Available,Not Available,Alert Data,Firewall,Netherlands,True,True,2,10,7,2023,Firefox,Mac OS X,Desktop
4,2023-07-16 13:11:00,71.166.185.76,189.243.174.238,6131,26646,TCP,1462,Data,DNS,Odit nesciunt dolorem nisi iste iusto. Animi v...,Not Available,0.52,Alert Triggered,DDoS,Known Pattern B,Blocked,Low,Dhanush Chad,Segment C,149.6.110.119,Not Available,Alert Data,Firewall,Belgium,True,True,16,13,7,2023,MSIE,Windows,Desktop


## Cyber Attacks on Companies

In [603]:
# 1. Checking for missing values
missing_values = df_comp.isnull().sum()
missing_values

Incident ID                        0
Company Name                       0
Industry                           0
Attack Type                        0
Date of Incident                   0
Breach Size (No. of Records)    6005
Financial Loss (USD)               0
Country                            0
Attack Vector                      0
Mitigation Time (Days)             0
dtype: int64

In [604]:
df_comp.head()

,Incident ID,Company Name,Industry,Attack Type,Date of Incident,Breach Size (No. of Records),Financial Loss (USD),Country,Attack Vector,Mitigation Time (Days)
0,1,White Inc,Retail,Ransomware,2018-08-27,6525846.0,506815.46,Italy,DDoS Attack,25
1,2,Vasquez PLC,Telecommunications,Ransomware,2018-05-24,9114896.0,1668001.94,Netherlands,Compromised Vendor,29
2,3,Gardner LLC,Retail,Supply Chain Attack,2018-07-04,3412886.0,4073566.14,Belgium,Compromised Vendor,22
3,4,Pena-Kirk,Manufacturing,Ransomware,2018-02-05,5829814.0,3211005.93,Belgium,Compromised Vendor,60
4,5,"Garcia, Butler and Kennedy",Healthcare,Ransomware,2019-04-05,9996201.0,4033042.07,UK,Compromised Vendor,59


In [605]:
# 2. Ensuring correct data types (e.g., 'Date of Incident' to datetime, numeric columns for breach size and financial loss)
df_comp['Date of Incident'] = pd.to_datetime(df_comp['Date of Incident'])
df_comp['Breach Size (No. of Records)'] = pd.to_numeric(df_comp['Breach Size (No. of Records)'])
df_comp['Financial Loss (USD)'] = pd.to_numeric(df_comp['Financial Loss (USD)'])
df_comp['Mitigation Time (Days)'] = pd.to_numeric(df_comp['Mitigation Time (Days)'])


In [606]:
# 3. Standardizing text data (Company Name, Country, and Industry columns)
df_comp['Company Name'] = df_comp['Company Name'].str.title().str.strip()
df_comp['Industry'] = df_comp['Industry'].str.title().str.strip()
df_comp['Country'] = df_comp['Country'].str.title().str.strip()


In [607]:
# 4. Removing duplicates
df_comp = df_comp.drop_duplicates()

In [608]:
# 5. Handling missing values for 'Breach Size (No. of Records)'
median_breach_size = df_comp['Breach Size (No. of Records)'].median()
df_comp['Breach Size (No. of Records)'].fillna(median_breach_size, inplace=True)

C:\Users\JasonGreich\AppData\Local\Temp\ipykernel_9944\381758125.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_comp['Breach Size (No. of Records)'].fillna(median_breach_size, inplace=True)


In [609]:
# 6. Creating new columns for year, month, and day from 'Date of Incident'
df_comp['Year'] = df_comp['Date of Incident'].dt.year
df_comp['Month'] = df_comp['Date of Incident'].dt.month
df_comp['Day'] = df_comp['Date of Incident'].dt.day


In [610]:
# Display the cleaned data
df_comp.head()

,Incident ID,Company Name,Industry,Attack Type,Date of Incident,Breach Size (No. of Records),Financial Loss (USD),Country,Attack Vector,Mitigation Time (Days),Year,Month,Day
0,1,White Inc,Retail,Ransomware,2018-08-27,6525846.0,506815.46,Italy,DDoS Attack,25,2018,8,27
1,2,Vasquez Plc,Telecommunications,Ransomware,2018-05-24,9114896.0,1668001.94,Netherlands,Compromised Vendor,29,2018,5,24
2,3,Gardner Llc,Retail,Supply Chain Attack,2018-07-04,3412886.0,4073566.14,Belgium,Compromised Vendor,22,2018,7,4
3,4,Pena-Kirk,Manufacturing,Ransomware,2018-02-05,5829814.0,3211005.93,Belgium,Compromised Vendor,60,2018,2,5
4,5,"Garcia, Butler And Kennedy",Healthcare,Ransomware,2019-04-05,9996201.0,4033042.07,Uk,Compromised Vendor,59,2019,4,5


# Active Users

In [611]:
# Convert the 'Date' column to datetime format
df_users['Date'] = pd.to_datetime(df_users['Date'])

In [612]:
# Create new columns for year, month, and day
df_users['Year'] = df_users['Date'].dt.year
df_users['Month'] = df_users['Date'].dt.month
df_users['Day'] = df_users['Date'].dt.day

In [613]:
df_users.head()

,Date,Active Online Users,Year,Month,Day
0,2018-01-01,412000000,2018,1,1
1,2018-01-02,413653325,2018,1,2
2,2018-01-03,412789947,2018,1,3
3,2018-01-04,413882307,2018,1,4
4,2018-01-05,412000000,2018,1,5


## Changing Columns Names

In [614]:
# Rename for individuals dataset
individuals_rename_mapping = {
    'Unnamed: 0': 'ID',
    'Timestamp': 'Timestamp',
    'Source IP Address': 'Source_IP_Address',
    'Destination IP Address': 'Destination_IP_Address',
    'Source Port': 'Source_Port',
    'Destination Port': 'Destination_Port',
    'Protocol': 'Protocol',
    'Packet Length': 'Packet_Length',
    'Packet Type': 'Packet_Type',
    'Traffic Type': 'Traffic_Type',
    'Payload Data': 'Payload_Data',
    'Malware Indicators': 'Malware_Indicators',
    'Anomaly Scores': 'Anomaly_Scores',
    'Alerts/Warnings': 'Alerts_Warnings',
    'Attack Type': 'Attack_Type',
    'Attack Signature': 'Attack_Signature',
    'Action Taken': 'Action_Taken',
    'Severity Level': 'Severity_Level',
    'User Information': 'User_Information',
    'Network Segment': 'Network_Segment',
    'Proxy Information': 'Proxy_Information',
    'Firewall Logs': 'Firewall_Logs',
    'IDS/IPS Alerts': 'IDS_IPS_Alerts',
    'Log Source': 'Log_Source',
    'Country': 'Country',
    'Valid Source IP': 'Valid_Source_IP',
    'Valid Destination IP': 'Valid_Destination_IP',
    'Day': 'Day',
    'Hour of Day': 'Hour_of_Day',
    'Month': 'Month',
    'Year': 'Year',
    'Browser': 'Browser',
    'Operating System': 'Operating_System',
    'Device Type': 'Device_Type'
}

# Rename for companies dataset
companies_rename_mapping = {
    'Unnamed: 0': 'ID',
    'Incident ID': 'Incident_ID',
    'Company Name': 'Company_Name',
    'Industry': 'Industry',
    'Attack Type': 'Attack_Type',
    'Date of Incident': 'Date_of_Incident',
    'Breach Size (No. of Records)': 'Breach_Size',
    'Financial Loss (USD)': 'Financial_Loss',
    'Country': 'Country',
    'Attack Vector': 'Attack_Vector',
    'Mitigation Time (Days)': 'Mitigation_Time',
    'Year': 'Year',
    'Month': 'Month',
    'Day': 'Day'
}

# Rename for active users dataset
active_users_rename_mapping = {
    'Unnamed: 0': 'ID',
    'Date': 'Date',
    'Active Online Users': 'Active_Online_Users',
    'Year': 'Year',
    'Month': 'Month',
    'Day': 'Day'
}

# Apply renaming
df_ind.rename(columns=individuals_rename_mapping, inplace=True)
df_comp.rename(columns=companies_rename_mapping, inplace=True)
df_users.rename(columns=active_users_rename_mapping, inplace=True)

# Save the updated files
df_ind.to_csv('./Transformed_Data/individuals_cyber_attacks_europe.csv', index=False)
df_comp.to_csv('./Transformed_Data/companies_cyber_attacks_europe.csv', index=False)
df_users.to_csv('./Transformed_Data/active_online_users_europe_2018_2023.csv', index=False)

# Loading 

## Loading to transfomation DB

In [616]:
# Database connection details
db_user = 'root'  # Replace with your MySQL username
db_password = 'root'  # Replace with your MySQL password
db_host = 'localhost'  # Or your database server hostname
db_name = 'transformed_cyber_staging'  # Replace with your database name
table_name = 'companies_cyber_attacks'  # Replace with your desired table name

# Create the database engine
engine = create_engine(f'mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}')

# Write the DataFrame to the MySQL table
df_comp.to_sql(table_name, con=engine, if_exists='replace', index=False)

print(f"Data imported successfully to table '{table_name}' in database '{db_name}'")

Data imported successfully to table 'companies_cyber_attacks' in database 'transformed_cyber_staging'


In [617]:
# Database connection details
db_user = 'root'  # Replace with your MySQL username
db_password = 'root'  # Replace with your MySQL password
db_host = 'localhost'  # Or your database server hostname
db_name = 'transformed_cyber_staging'  # Replace with your database name
table_name = 'individual_cyber_attacks'  # Replace with your desired table name

# Create the database engine
engine = create_engine(f'mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}')

# Write the DataFrame to the MySQL table
df_ind.to_sql(table_name, con=engine, if_exists='replace', index=False)

print(f"Data imported successfully to table '{table_name}' in database '{db_name}'")

Data imported successfully to table 'individual_cyber_attacks' in database 'transformed_cyber_staging'


In [618]:
# Database connection details
db_user = 'root'  # Replace with your MySQL username
db_password = 'root'  # Replace with your MySQL password
db_host = 'localhost'  # Or your database server hostname
db_name = 'transformed_cyber_staging'  # Replace with your database name
table_name = 'active_online_users'  # Replace with your desired table name

# Create the database engine
engine = create_engine(f'mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}')

# Write the DataFrame to the MySQL table
df_users.to_sql(table_name, con=engine, if_exists='replace', index=False)

print(f"Data imported successfully to table '{table_name}' in database '{db_name}'")

Data imported successfully to table 'active_online_users' in database 'transformed_cyber_staging'


## Loading To warehouse

In [ ]:
# Example assuming 'Timestamp' column exists in both dataframes
df_date = pd.concat([df_ind['Timestamp'], df_comp['Date_of_Incident'], df_users['Date']]).drop_duplicates()


# Convert to datetime
df_date = pd.to_datetime(df_date)

# Create DateDimension table
df_date_dimension = pd.DataFrame({
    'Day': df_date.dt.day,
    'Month': df_date.dt.month,
    'Year': df_date.dt.year,
})
df_date_dimension.sort_values(by=['Year', 'Month', 'Day'], inplace=True)
df_date_dimension.reset_index(drop=True, inplace=True)
df_date_dimension['DateID'] = df_date_dimension.index + 1

In [ ]:
# Combine countries from individual and company dataframes and drop duplicates
df_country = pd.concat([df_ind['Country'], df_comp['Country']]).drop_duplicates()

# Create CountryDimension table
df_country_dimension = pd.DataFrame({
    'Country': df_country
})
df_country_dimension.reset_index(drop=True, inplace=True)
df_country_dimension['CountryID'] = df_country_dimension.index + 1


In [ ]:
# Extract IP-related columns from df_ind and drop duplicates
df_detailed_location_dimension = df_ind[['Source_IP_Address', 'Destination_IP_Address', 'Valid_Source_IP', 'Valid_Destination_IP']].drop_duplicates()


df_detailed_location_dimension['DetailedGeoLocationID'] = df_detailed_location_dimension.index + 1

In [ ]:
# Extract network traffic columns from df_ind and drop duplicates
df_network_traffic_dimension = df_ind[['Source_Port', 'Destination_Port', 'Protocol', 'Packet_Length', 'Packet_Type', 'Traffic_Type', 'Network_Segment']].drop_duplicates()


df_network_traffic_dimension['NetworkTrafficID'] = df_network_traffic_dimension.index + 1

In [ ]:
# Extract company-related columns from df_comp and drop duplicates
df_company_dimension = df_comp[['Company_Name', 'Industry']].drop_duplicates()

df_company_dimension['CompanyID'] = df_company_dimension.index + 1

In [ ]:
# Extract device-related columns from df_ind and drop duplicates
df_device_dimension = df_ind[['Browser', 'Operating_System', 'Device_Type']].drop_duplicates()


df_device_dimension['DeviceID'] = df_device_dimension.index + 1

In [ ]:
df_ind.rename(columns={'IDS/IPS_Alerts': 'IDS_IPS_Alerts'}, inplace=True)
df_ind.rename(columns={'Alerts/Warnings': 'Alerts_Warnings'}, inplace=True)

df_individual_facts = df_ind[['Browser', 'Operating_System', 'Device_Type','Source_Port', 'Destination_Port', 'Protocol', 'Packet_Length', 'Packet_Type', 'Traffic_Type', 'Network_Segment','Source_IP_Address', 'Destination_IP_Address','Country','Attack_Type', 'Payload_Data', 'Malware_Indicators', 'Anomaly_Scores', 'Severity_Level', 'Alerts_Warnings', 'Attack_Signature', 'Action_Taken', 'User_Information', 'Proxy_Information', 'Firewall_Logs', 'IDS_IPS_Alerts', 'Log_Source']]


In [ ]:
df_comp.rename(columns={'Breach_Size_(No._of_Records)': 'Breach_Size_No_of_Records'}, inplace=True)
df_comp.rename(columns={'Mitigation_Time_(Days)': 'Mitigation_Time_Days'}, inplace=True)
df_comp.rename(columns={'Financial_Loss_(USD)': 'Financial_Loss_USD'}, inplace=True)

df_company_facts = df_comp[['Company_Name', 'Industry','Country','Attack_Type', 'Breach_Size_No_of_Records', 'Attack_Vector', 'Mitigation_Time_Days', 'Financial_Loss_USD']]


In [ ]:
df_active_users_facts = df_users[['Active_Online_Users']]


In [ ]:
# Merge DateID into the individual facts
df_individual_facts = pd.merge(df_individual_facts, df_date_dimension, how='left', 
                               left_on=pd.to_datetime(df_ind['Timestamp']).dt.date,
                               right_on=pd.to_datetime(df_date_dimension[['Year', 'Month', 'Day']]).dt.date)

df_individual_facts['DateID'] = df_individual_facts['DateID']

# Merge DateID into the company facts
df_company_facts = pd.merge(df_company_facts, df_date_dimension, how='left', 
                            left_on=pd.to_datetime(df_comp['Date_of_Incident']).dt.date,
                            right_on=pd.to_datetime(df_date_dimension[['Year', 'Month', 'Day']]).dt.date)

df_company_facts['DateID'] = df_company_facts['DateID']

# Merge DateID into the active users facts
df_active_users_facts = pd.merge(df_active_users_facts, df_date_dimension, how='left', 
                                 left_on=pd.to_datetime(df_users['Date']).dt.date,
                                 right_on=pd.to_datetime(df_date_dimension[['Year', 'Month', 'Day']]).dt.date)

df_active_users_facts['DateID'] = df_active_users_facts['DateID']


In [ ]:
print(df_individual_facts.columns)
print(df_country_dimension.columns)


In [ ]:
# Merge CountryID into the individual facts
df_individual_facts = pd.merge(df_individual_facts, df_country_dimension, how='left', 
                               left_on='Country', right_on='Country')

df_individual_facts['CountryID'] = df_individual_facts['CountryID']

# Merge CountryID into the company facts
df_company_facts = pd.merge(df_company_facts, df_country_dimension, how='left', 
                            left_on='Country', right_on='Country')

df_company_facts['CountryID'] = df_company_facts['CountryID']


In [ ]:
# Merge DetailedGeoLocationID into the individual facts
df_individual_facts = pd.merge(df_individual_facts, df_detailed_location_dimension, how='left',
                               left_on=['Source_IP_Address', 'Destination_IP_Address'], 
                               right_on=['Source_IP_Address', 'Destination_IP_Address'])

df_individual_facts['DetailedGeoLocationID'] = df_individual_facts['DetailedGeoLocationID']


In [ ]:
# Merge NetworkTrafficID into the individual facts
df_individual_facts = pd.merge(df_individual_facts, df_network_traffic_dimension, how='left',
                               left_on=['Source_Port', 'Destination_Port', 'Protocol', 'Packet_Length', 'Packet_Type', 'Traffic_Type', 'Network_Segment'],
                               right_on=['Source_Port', 'Destination_Port', 'Protocol', 'Packet_Length', 'Packet_Type', 'Traffic_Type', 'Network_Segment'])

df_individual_facts['NetworkTrafficID'] = df_individual_facts['NetworkTrafficID']


In [ ]:
# Merge DeviceID into the individual facts
df_individual_facts = pd.merge(df_individual_facts, df_device_dimension, how='left', 
                               left_on=['Browser', 'Operating_System', 'Device_Type'], 
                               right_on=['Browser', 'Operating_System', 'Device_Type'])

df_individual_facts['DeviceID'] = df_individual_facts['DeviceID']


In [ ]:
# Merge CompanyID into the company facts
df_company_facts = pd.merge(df_company_facts, df_company_dimension, how='left', 
                            left_on=['Company_Name', 'Industry'], 
                            right_on=['Company_Name', 'Industry'])

df_company_facts['CompanyID'] = df_company_facts['CompanyID']


In [ ]:
df_individual_facts.drop(columns=['Browser', 'Operating_System', 'Device_Type','Source_Port', 'Destination_Port', 'Protocol', 'Packet_Length', 'Packet_Type', 'Traffic_Type', 'Network_Segment','Source_IP_Address', 'Destination_IP_Address','Country'], inplace=True)
df_company_facts.drop(columns=['Company_Name', 'Industry','Country'], inplace=True)

In [ ]:
# Define your database connection parameters
db_user = 'root'  # Replace with your MySQL username
db_password = 'root'  # Replace with your MySQL password
db_host = 'localhost'  # Or your database server hostname
db_name = 'cyber_attacks_warehouse'  # Replace with your database name

# Create the connection string
connection_string = f'mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}'

# Create a connection to the database
engine = create_engine(connection_string)
# Load each dataframe into the respective SQL tables
df_date_dimension.to_sql('datedimension', engine, if_exists='append', index=False)



In [ ]:
df_country_dimension.to_sql('countrydimension', engine, if_exists='append', index=False)



In [ ]:
df_detailed_location_dimension.to_sql('detailedlocationdimension', engine, if_exists='append', index=False)
df_network_traffic_dimension.to_sql('networktrafficdimension', engine, if_exists='append', index=False)
df_company_dimension.to_sql('companydimension', engine, if_exists='append', index=False)
df_device_dimension.to_sql('devicedimension', engine, if_exists='append', index=False)


df_individual_facts.to_sql('individualcyberattacksfact', engine, if_exists='append', index=False)
df_company_facts.to_sql('companiescyberattacksfact', engine, if_exists='append', index=False)
df_active_users_facts.to_sql('activeonlineusersfact', engine, if_exists='append', index=False)